# Data Science Jobs Analyze


Group:
* Anchal
* Angela
* Larissa
* Fabio
* Felipe

This program extracts data from Indeed.ca website to analyze main caracteristics 
of Data Science Job opportunities in Toronto

In [69]:
#################### IMPORT LIBRARIES ####################

In [70]:
#import bs4
import numpy as np
import pandas as pd
import re 
import requests
import datetime
from bs4 import BeautifulSoup

In [71]:
#################### SEARCH ARGUMENTS ####################

In [72]:
input_job = "Data Scientist"
input_quote = False # add quotation marks("") to your input_job
input_city = "" # leave empty if input_city is not specified
input_state = "Canada"
sign = "+"
BASE_URL_indeed =  'https://ca.indeed.com/' #'http://www.indeed.com'


In [73]:
##### Function for Transform searching keywords #####
def transform(input,sign, quote = False):
    syntax = input.replace(" ", sign)
    if quote == True:
        syntax = ''.join(['"', syntax, '"'])
    return(syntax)

In [74]:
################# GENERATING BASE INDEED URL #################

In [75]:

if not input_city: # if (input_city is "")
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', input_state]
    url_indeed = ''.join(url_indeed_list)
else: # input_city is not ""
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', transform(input_city, sign), '%2C+', input_state]
    url_indeed = ''.join(url_indeed_list)

print(url_indeed)


https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada


In [76]:
# get the HTML code from the URL
rawcode_indeed = requests.get(url_indeed)

# Choose "lxml" as parser
soup_indeed = BeautifulSoup(rawcode_indeed.text, "lxml")


In [77]:
# total number of results
num_total_indeed = soup_indeed.find(
                        id = 'searchCount').contents[0].split()[-2]

print("total results:")
print(soup_indeed.find(id = 'searchCount').contents[0])
print(soup_indeed.find(id = 'searchCount').contents[0].split()[-2])
num_total_indeed = re.sub("[^0-9]","", num_total_indeed) # remove non-numeric characters in the string
num_total_indeed = int(num_total_indeed)
#print(num_total_indeed)

# total number of pages
num_pages_indeed = int(np.ceil(num_total_indeed/10.0))
#print(num_pages_indeed)

# create an empty dataframe
df_base = pd.DataFrame()
# the date for today
now = datetime.datetime.now()
now_str = now.strftime("%m/%d/%Y")
now_str_name=now.strftime('%m%d%Y')

total results:

        Page 1 of 1,039 jobs
1,039


In [78]:
############ function to clean text before scraping ########### 

In [79]:
def text_cleaner(dirty_text):

    text = dirty_text

    lines = (line.strip() for line in text.splitlines()) # break into lines

    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    
    # Now clean out all of the unicode junk (this line works great!!!)
    
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception
   
    #print(text)
    text = text.decode('utf-8')
    text = re.sub("[^a-zA-Z+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                             # Also include + for C++
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    
    text = text.lower().split()  # Go to lower case and split them apart
    
    
   # stop_words = set(stopwords.words("english")) # Filter out any stop words
   # text = [w for w in text if not w in stop_words]
    
    text = list(set(text)) # Last, just get the set of these. Ignore counts (we are just looking at whether a term existed
                           # or not on the website)
    
    return text

######### CREATING DATA FRAME WITH ALL JOB OPPORTUNITIES URLS #####

In [80]:
######## functions ###########
def Salary(soup):
    try:
        salary = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item'})
        return  salary.contents
    except:
        return 'Not Informed'
    
def Job_Type(soup):
    try:
        rawcode = requests.get(url)
        soup = BeautifulSoup(rawcode.text, "html.parser")
        job_type = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs'})
        return  job_type.contents
    except:
        return 'Not Informed'


In [81]:
#<a target="" id="sja2" data-tn-element="jobTitle" 
#class="jobtitle turnstileLink visited" href="https://ca.indeed.com/pagead/clk?mo=r&amp;ad=-6NYlbfkN0DfaJzxeKsHg3yNa09Lu8cQQduvjegwzxl5yjE4sdO8aCQ7r433WSCgz7HcYFllrIChqTJMS6QCmfssNe4X0QSHiOgrDwcCGBAYCZvI_BwQi7xLmaCBM0jiSrhcAf8rdymPFkMDSwpaPlZeI6fWymNruny32jwnnByw7jFGzo9NkW4X4yQ_PjrCR0hXfYjXumiRywYP9ZFPwJ05-BsjrYICoWayMmyV5luzrs47wlt6XiU39YnQnHLOmYEku1S_tF_og5cAF72x2FLAu4Jj5cF3wfBBNlfY9o4vvFCQ2zX8xZsaIZs8MZT600XjvA3vKNlcZ9fBxcQlN7V6ZXqyEyGaTPd1skBYqY1xerFEg8mS6V0Nds-l34VWELv7tXYxQ_-Xknjhui6qMm34NSRGlZLzJTzhh6WHe5XKDpRZgpQgxzLNcEewWW5x_4uDad7Z5RGOuHBqD3DBBrzOl4tFTt2aWNmlitCCXgGl0nQqh0zJc6aR5aiTUNMA3RAdxd1GCRaviSxDjc2I2qYiPSdz36Ts&amp;vjs=3&amp;p=2&amp;sk=&amp;fvj=0&amp;tk=1csfpbj6050so803&amp;jsa=240&amp;sal=0&amp;sal=0&amp;oc=1&amp;sal=0" title="Data Scientist" rel="noopener nofollow" onmousedown="sjomd('sja2'); clk('sja2');" 
#onclick="setRefineByCookie([]); sjoc('sja2',0); convCtr('SJ')"><b>Data</b> <b>Scientist</b></a>

In [140]:
########################################
##### Loop for all the total pages #####
########################################

num_pages_indeed = 1 
df_base = pd.DataFrame()

for i in range(1, num_pages_indeed+1):
    # generate the URL
    url = ''.join([url_indeed, '&start=', str(i*10)])
    print(url)

    # get  LXML
    rawcode = requests.get(url)
    soup_lxml = BeautifulSoup(rawcode.text, "lxml")
    
    
    #Extracting the Salary


    # pick out all the "div" with "class="job-row"
    divs = soup_lxml.findAll("div")
    job_divs = [jp for jp in divs if not jp.get('class') is None
                    and 'row' in jp.get('class')]

    count = 1
    for job in job_divs:
        try:
            # job id
            id = job.get('data-jk', None)
            print(id)
            
            # job link related to job id
            link = BASE_URL_indeed + '/viewjob'+ '?jk=' + id
            #link = 'https://ca.indeed.com/Data-Scientist-jobs-in-Canada?vjk=' + id
            print(link)
            
            # job title
            title = job.find('a', attrs={'data-tn-element': 'jobTitle'}).attrs['title']
            print(title)
            
            #sja = 'sja'+ str(count)
            #href = BASE_URL_indeed + job.find('a', attrs={'id': sja}).attrs['href']
            
            
            #print(href)
            #count = count + 1
            # job company
            company = job.find('span', {'class': 'company'}).text.strip()
            print(company)
            
            # job location
            location = job.find('span', {'class': 'location'}).text.strip()
            print(location)
            #city, province
            city,province = location.split(',')
            
            ### html.parser - Felipe
            r = requests.get(link)
            soup_html = BeautifulSoup(r.text, "html.parser")
            #print(soup_html)
            
            # salary
            salary = Salary(soup_html)
            print('salary:' + salary)
            
            # job type
            jobtype = Job_Type(soup_html)

            
            #Extracting the Job Title
            position = soup_html.find(attrs = {'class' : 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
            
            
           
            # job Requirements
            try:   
                #page = requests.get(link)
                #soup = BeautifulSoup(page.text, 'html.parser')
                text  = soup_html.get_text()
                jobrequirements = text
                #jobrequirements = text_cleaner(text)
                
            except:
                print('exception 1')
                jobrequirements = 'Forbidden'

            
        except:
            continue
        print('  *** Job ' + id + ' has been appended in the Data Frame ***')
        df_base = df_base.append(dict({'Job_ID': id,
                                  'JobTitle': title,
                                  'JobType':jobtype,
                                  'CompanyName': company,
                                  'Salary' : salary ,
                                  'Position':position,
                                  'City': city,
                                  'Province':province,
                                  'JobRequirements':jobrequirements,
                                  'Date': now_str,
                                  'From':"Indeed",
                                  'JobLink':link}),ignore_index=True)
        
print('Finished...')
print(df_base.head())


https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=10
61c9aaeee416a1c6
https://ca.indeed.com//viewjob?jk=61c9aaeee416a1c6
Data Scientist
Sun Life Financial
4ca0cac253b0f9d2
https://ca.indeed.com//viewjob?jk=4ca0cac253b0f9d2
Data Scientist
SPORTLOGiQ
631aecbd912f8389
https://ca.indeed.com//viewjob?jk=631aecbd912f8389
Data Scientist, Algorithms & Data Systems
Boxy Charm
e5f3946130fe18b0
https://ca.indeed.com//viewjob?jk=e5f3946130fe18b0
Customer & Energy Data Scientist
BC Hydro
0271b3456b2ff5d9
https://ca.indeed.com//viewjob?jk=0271b3456b2ff5d9
Data Scientist
ATB Financial
Calgary, AB
e22404d0acf974af
https://ca.indeed.com//viewjob?jk=e22404d0acf974af
Data Scientist
Quartic.ai
Oakville, ON
3625131c23ab13f2
https://ca.indeed.com//viewjob?jk=3625131c23ab13f2
Data Scientist
Xylem
Mississauga, ON
salary:Not Informed
  *** Job 3625131c23ab13f2 has been appended in the Data Frame ***
86e0e9a1dab7acf3
https://ca.indeed.com//viewjob?jk=86e0e9a1dab7acf3
Data Scientist
Flipp
Toronto, ON
s

In [99]:

df_base.to_csv('out.csv')

In [100]:
############# READING DATAFRAME AND SCRAPING JOB DESCRIPTION ############

In [101]:
#df_base.JobRequirements.values

In [111]:
skill_source = pd.read_excel('ds.xlsx', sheet_name='skill')


kw = pd.read_excel('ds_kw.xlsx')

print(skill_source)
print(kw)


   SkillID           Skill       Category
0        1          Python           Hard
1        2             SQL           Hard
2        3         BigData           Hard
3        4           NumPy           Hard
4        5          Spring           Hard
5        6      Mathematcs      Education
6        7             SQL           Hard
7        8      Statistics      Education
8       -1  Not Identified  Not Idenified
   kwID      KeyWord        JobTitle  Skill_ID
0     1  mathematics  Data Scientist         6
1     2         Math  Data Scientist         6
2     3       python  Data Scientist         1
3     4     big data  Data Scientist         3
4     5      bigdata  Data Scientist         3
5     6          sql  Java Developer         7
6     7       spring  Java Developer         5
7     8    Not match  Data Scientist        -1


In [103]:

# Functions
def find_skill(skill, jobReq ):
    if skill in jobReq:
        return True
    else:
        return False

In [104]:
df_skill_aux = pd.DataFrame(columns=['Job_ID', 'KeyWord', 'Skill_ID'])


In [105]:
df_skill_aux

,Job_ID,KeyWord,Skill_ID


In [138]:
z = 0
for index,row in df_base.iterrows():
    k = kw.JobTitle == row["JobTitle"], ['KeyWord', 'Skill_ID']
    jobReq = row["JobRequirements"]
    print(k)
    job_ID = row["Job_ID"]
    print(job_ID)
    
    #for idx, skills in k.iterrows():
    for skills in k:
        skill = skills["KeyWord"]
        print(skill)
        skill_ID = skills["Skill_ID"]
        func = find_skill(skill, jobReq)
        print(func)
        if func:
            print(job_ID, skill, skill_ID)
            df_skill_aux.loc[z] = [job_ID, skill, skill_ID]
            z += 1

(0     True
1     True
2     True
3     True
4     True
5    False
6    False
7     True
Name: JobTitle, dtype: bool, ['KeyWord', 'Skill_ID'])
3625131c23ab13f2


KeyError: 'KeyWord'

In [97]:
df_skill_aux

,Job_ID,KeyWord,Skill_ID
0,3625131c23ab13f2,mathematics,6
1,3625131c23ab13f2,python,1
2,79f244142735e51b,python,1
3,9c977136e5f64e8a,python,1
4,85db0c4b9dd0d204,mathematics,6
5,d4a402329a3013c6,mathematics,6
6,e1de8b679753f22a,python,1
7,bdf8400b27b22160,mathematics,6
8,bdf8400b27b22160,python,1
9,2172d8fe11ab66ae,python,1


In [98]:
# Creating df_tmp
df_tmp = pd.DataFrame()


In [ ]:
#df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='ID')
df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='Job_ID')
df_tmp.head()

In [100]:
############# Treating null values #################
values = {'Skill_ID': '-1', 'KeyWord': "Not found"}
df_tmp = df_tmp.fillna(value=values)

In [101]:
df_tmp['Skill_ID'] = df_tmp['Skill_ID'].astype(int)

In [102]:
skill_source

,SkillID,Skill,Category
0,1,Python,Hard
1,2,SQL,Hard
2,3,BigData,Hard
3,4,NumPy,Hard
4,5,Spring,Hard
5,6,Mathematcs,Education
6,7,SQL,Hard
7,8,Statistics,Education
8,-1,Not Identified,Not Idenified


In [106]:
df_jobs = df_tmp.merge(skill_source, left_on='Skill_ID', right_on='Skill_ID', how='left')


KeyError: 'Skill_ID'

In [104]:
df_jobs

NameError: name 'df_jobs' is not defined

In [ ]:
df_jobs = df_jobs.drop_duplicates(subset={'Job_ID','Skill_ID'}, keep='first', inplace=False)

In [ ]:
df_jobs